In [47]:
import pandas as pd
import numpy as np
df=pd.read_csv("complete_dataset_V2.csv")
df['rideable_type'] = df['rideable_type'].replace('docked_bike', 'classic_bike')
#drop columns
new_df=df.drop(columns=["ride_id",
                        "started_at","ended_at",
                        "start_station_name","end_station_name",
                        "start_station_id", "end_station_id",
                        "snow","wpgt","tsun"])
new_df=new_df.dropna()
#check duplicates
new_df[new_df.duplicated()].shape
#exclude trips that are shoter than 1 min
new_df=new_df[new_df["trip_time"]>=1]

#create dummy
categorical_columns = ["member_casual", "rideable_type", 
                       "month","wind_direction", "day_of_the_week","hour","start_station_cluster","end_station_cluster"]

dummies = pd.get_dummies(new_df[categorical_columns], drop_first=True)
# Dropping the original categorical columns from the DataFrame
new_df = new_df.drop(columns=categorical_columns)
# Concatenating the dummy variables to the original DataFrame
new_df = pd.concat([new_df, dummies], axis=1)

#Exclude average_speed columns that has infinity values from the dataset
new_df = new_df[new_df['average_speed'] != np.inf]

# new_df.to_csv("pre-processed.csv")

/var/folders/_t/yhfwr7zs235_z88czy6gmp2m0000gn/T/ipykernel_5992/3551136494.py:3: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("complete_dataset_V2.csv")


In [48]:
#standardization if needed
from sklearn.preprocessing import MinMaxScaler
import numpy as np


# Exclude "average_speed" from scaling
columns_to_scale = new_df.columns.difference(['average_speed',"start_station_cluster", "end_station_cluster"])
X_to_scale = new_df[columns_to_scale]

# Apply MinMaxScaler to the other columns
scaler = MinMaxScaler()
X_std = scaler.fit_transform(X_to_scale)
X_std = pd.DataFrame(X_std, columns=columns_to_scale)

# Add the "average_speed" column back to the scaled DataFrame
X_std['average_speed'] = new_df['average_speed'].values
X_std['start_station_cluster'] = new_df['start_station_cluster'].values
X_std['end_station_cluster'] = new_df['end_station_cluster'].values

# Now X_std has the scaled values for all columns except "average_speed"

In [49]:
X_std.to_csv("pre-processed.csv")